### Carga de datos, etc

In [24]:
import os
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2
import PIL.Image as Image

from skimage import io, color, transform
from skimage.feature import hog

import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

In [25]:
gray_matrix = np.load('DatosDF\gray_matrix.npy', allow_pickle=True)
pca = np.load('DatosDF\pca.npy', allow_pickle=True)
estados = np.load('DatosDF\estados.npy', allow_pickle=True)
label = np.load('DatosDF\labels.npy', allow_pickle=True)

<>:1: SyntaxWarning: invalid escape sequence '\g'
<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:3: SyntaxWarning: invalid escape sequence '\e'
<>:4: SyntaxWarning: invalid escape sequence '\l'
<>:1: SyntaxWarning: invalid escape sequence '\g'
<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:3: SyntaxWarning: invalid escape sequence '\e'
<>:4: SyntaxWarning: invalid escape sequence '\l'
C:\Users\gonza\AppData\Local\Temp\ipykernel_7336\3099182347.py:1: SyntaxWarning: invalid escape sequence '\g'
  gray_matrix = np.load('DatosDF\gray_matrix.npy', allow_pickle=True)
C:\Users\gonza\AppData\Local\Temp\ipykernel_7336\3099182347.py:2: SyntaxWarning: invalid escape sequence '\p'
  pca = np.load('DatosDF\pca.npy', allow_pickle=True)
C:\Users\gonza\AppData\Local\Temp\ipykernel_7336\3099182347.py:3: SyntaxWarning: invalid escape sequence '\e'
  estados = np.load('DatosDF\estados.npy', allow_pickle=True)
C:\Users\gonza\AppData\Local\Temp\ipykernel_7336\3099182347.py:4: SyntaxWarning: inv

In [26]:
df = pd.DataFrame({'gray_matrix': gray_matrix,'pca': pca,'estados': estados,'label': label})

In [27]:
dfxusar = df[df['estados'] == 1]
elresto = df[df['estados'] == 0]
dfxusar

,gray_matrix,pca,estados,label
16,"[[175, 166, 185, 198, 202, 213, 212, 209, 210,...","[1.0912320462992262, 0.3657536288748723, 0.283...",1,angry
23,"[[26, 29, 30, 35, 45, 60, 108, 145, 161, 161, ...","[-0.7706739587282956, 0.721693515528713, 1.215...",1,angry
30,"[[17, 18, 19, 19, 17, 15, 16, 17, 18, 20, 18, ...","[0.7941313890964633, -0.6699144967452324, 0.43...",1,angry
31,"[[0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 1, 13, 37, ...","[1.5112234099411108, 1.0339331627332178, -0.16...",1,angry
45,"[[254, 254, 254, 254, 254, 254, 254, 254, 254,...","[1.0902360530996267, 0.01650497232340127, -0.2...",1,angry
...,...,...,...,...
35863,"[[9, 38, 69, 99, 86, 102, 111, 105, 65, 58, 64...","[-0.5491822800503183, 0.37287554580139537, 0.5...",1,surprise
35869,"[[253, 253, 254, 253, 170, 101, 105, 102, 85, ...","[-0.831189505684772, -0.9824639195355812, -0.5...",1,surprise
35875,"[[252, 247, 146, 93, 94, 89, 99, 94, 93, 83, 9...","[-1.6612481070642713, -0.7122384048718365, 0.9...",1,surprise
35878,"[[33, 44, 49, 40, 35, 28, 23, 14, 9, 16, 17, 5...","[0.4970099444085885, -0.026777335969787118, 0....",1,surprise


In [28]:
X = np.array(dfxusar['pca'].values.tolist())
y = dfxusar['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19011)

## RandomForest

In [29]:
%%time
# Crear el modelo
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo
rf_classifier.fit(X_train, y_train)

CPU times: total: 16.3 s
Wall time: 19.2 s


RandomForestClassifier(random_state=42)

In [30]:
%%time
y_pred_e= rf_classifier.predict(X_train)
accuracy = accuracy_score(y_train, y_pred_e)
print(f'Precisión del modelo: {accuracy}')

print(classification_report(y_train, y_pred_e))

Precisión del modelo: 1.0
              precision    recall  f1-score   support

       angry       1.00      1.00      1.00       462
     disgust       1.00      1.00      1.00       435
        fear       1.00      1.00      1.00       579
       happy       1.00      1.00      1.00       650
     neutral       1.00      1.00      1.00       622
         sad       1.00      1.00      1.00       467
    surprise       1.00      1.00      1.00       473

    accuracy                           1.00      3688
   macro avg       1.00      1.00      1.00      3688
weighted avg       1.00      1.00      1.00      3688

CPU times: total: 141 ms
Wall time: 163 ms


In [31]:
%%time
# Realizar predicciones en el conjunto de prueba
y_pred = rf_classifier.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy}')

print(classification_report(y_test, y_pred))

Precisión del modelo: 0.7215601300108342
              precision    recall  f1-score   support

       angry       0.80      0.67      0.73       101
     disgust       0.75      0.45      0.56       112
        fear       0.64      0.71      0.67       156
       happy       0.68      0.95      0.79       149
     neutral       0.73      0.86      0.79       153
         sad       0.73      0.61      0.66       119
    surprise       0.83      0.70      0.76       133

    accuracy                           0.72       923
   macro avg       0.74      0.71      0.71       923
weighted avg       0.73      0.72      0.71       923

CPU times: total: 31.2 ms
Wall time: 41.1 ms


In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

# Obtener los árboles individuales en el bosque
trees = rf_classifier.estimators_

# Calcular la profundidad máxima y promedio de los árboles
max_depths = [tree.tree_.max_depth for tree in trees]
average_depth = sum(max_depths) / len(max_depths)

print(f"Profundidad máxima de los árboles en el Random Forest: {max(max_depths)}")
print(f"Profundidad promedio de los árboles en el Random Forest: {average_depth}")

Profundidad máxima de los árboles en el Random Forest: 30
Profundidad promedio de los árboles en el Random Forest: 22.17


In [45]:
# Obtener los valores de min_samples_leaf de los árboles
min_samples_leaf_values = [tree.min_samples_leaf for tree in trees]

# Calcular la cantidad mínima de min_samples_leaf en los árboles
min_samples_leaf_min = min(min_samples_leaf_values)

# Calcular la cantidad promedio de min_samples_leaf en los árboles
min_samples_leaf_average = sum(min_samples_leaf_values) / len(min_samples_leaf_values)

print(f"Valor mínimo de min_samples_leaf en los árboles del Random Forest: {min_samples_leaf_min}")
print(f"Valor promedio de min_samples_leaf en los árboles del Random Forest: {min_samples_leaf_average}")

Valor mínimo de min_samples_leaf en los árboles del Random Forest: 1
Valor promedio de min_samples_leaf en los árboles del Random Forest: 1.0


In [40]:
print('numero de atributos')
len(df['pca'][0])

numero de atributos


821

## [Conjunto Pequeño] GridSearch - Optimizacion de parametros

### Primero

In [ ]:
%%time

# Define el modelo de Random Forest
rf_classifier = RandomForestClassifier()

# Define una cuadrícula de hiperparámetros que deseas explorar
param_grid = {
    'max_depth': [15,30, None],      
    'min_samples_split': [5,15,30],     
    'min_samples_leaf': [5,15,30],       
    'max_features' : [5,20,'sqrt','log2'],
    'n_estimators' : [300,600,900],
    'criterion': ['gini']
}

# Realiza la búsqueda en cuadrícula con validación cruzada
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='accuracy', verbose=10,n_jobs=12, return_train_score = True)
grid_search.fit(X_train, y_train)

# Muestra los mejores hiperparámetros encontrados
best_params = grid_search.best_params_
print(f"Mejores hiperparámetros: {best_params}")

# Entrena el modelo con los mejores hiperparámetros en todo el conjunto de entrenamiento
best_rf_classifier = grid_search.best_estimator_
best_rf_classifier.fit(X_train, y_train)

# Evalúa el modelo en el conjunto de prueba
y_pred = best_rf_classifier.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy}')

print(classification_report(y_test, y_pred))

**Resultados** 

Precisión del modelo: 0.8266522210184182
- 'max_depth': [15,30, None],      
- 'min_samples_split': [5,15,30],     
- 'min_samples_leaf': [5,15,30],       
- 'max_features' : [5,20,'sqrt','log2'],
- 'n_estimators' : [300,600,900],
- 'criterion': ['gini']